In [ ]:
!git clone https://github.com/gicait/plitterstreet.git
!pip install --upgrade ../.
# # exit(0)


In [ ]:
!pwd

In [ ]:
# download a sample video from https://drive.google.com/file/d/1g5f1eL7iRRW1I99mJwV8ah_le9RH9aY0/view?usp=sharing
# ignore downloading and modify path in the below cells, if you already have a video and gps data in csv file

!mkdir /media/ryzen/DATA/scripts/tests/data
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1g5f1eL7iRRW1I99mJwV8ah_le9RH9aY0' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1g5f1eL7iRRW1I99mJwV8ah_le9RH9aY0" -O /media/ryzen/DATA/scripts/tests/data/sample.zip && rm -rf /tmp/cookies.txt
!unzip /media/ryzen/DATA/scripts/tests/data/sample.zip -d /media/ryzen/DATA/scripts/tests/data/.
!rm -rf '/media/ryzen/DATA/scripts/tests/data/sample.zip'

In [ ]:
from plitterstreet.video import Video

In [ ]:
vid = Video('/media/ryzen/DATA/scripts/tests/data/sample_video.mp4', '/media/ryzen/DATA/scripts/tests/data/sample_GPS5.csv')
vid.ExtractbyGps()

In [ ]:
from plitterstreet.plitter_street import StreetData

In [ ]:
street = StreetData('/media/ryzen/DATA/scripts/tests/data/sample_video')

In [ ]:
street.images_list

In [ ]:
street.load_images_from_dir()

# sorted(street.images, key = lambda i: i['name'])

In [ ]:
street.images

In [ ]:
from plitterstreet.predict import EdgePredictor

In [ ]:
predictor = EdgePredictor('automl_edge')

In [ ]:
street.predict_on_images(predictor, min_score_thr=0.25, dst_path='/media/ryzen/DATA/scripts/tests/data/sample_video')

In [ ]:
street.predictions

In [ ]:
street.save()

In [ ]:
street.load_data_from_json()

In [ ]:
# import folium
# m = folium.Map(tiles="OpenStreetMap")

# images = folium.FeatureGroup(name='Images',overlay=True)
# hotline = folium.FeatureGroup(name='Hot line',overlay=True)
# heatmap = folium.FeatureGroup(name='Heat map',overlay=True)
# piles = folium.FeatureGroup(name='Piles',overlay=True)
# trashbins = folium.FeatureGroup(name='Trash bins',overlay=True)

# images.add_to(m)
# hotline.add_to(m)
# heatmap.add_to(m)
# piles.add_to(m)
# trashbins.add_to(m)

# folium.LayerControl().add_to(m)

m = street.draw_street_on_map()
m

In [ ]:
m.save(street.images_dir+'/sample_video.html')

In [ ]:
import os
from plitterstreet.plitter_street import StreetData

class City:
    
    def __init__(self, path_to_videos, path_to_frames=None, path_to_predictions=None):
        if os.path.isdir(path_to_videos):
            videos = list(filter(lambda f: f.lower().endswith(('.mp4', '.avi')), os.listdir(path_to_videos)))
            if len(videos) > 1:
                self.path = os.path.abspath(path_to_videos)
                self.videos = videos
        else:
            print('Invalid path to videos')
            return None
        
        if path_to_frames and os.path.isdir(path_to_frames):
            self.path_to_frames = os.path.abspath(path_to_frames)
        else:
            self.path_to_frames = self.path
            print('path to frames is invalid, choosing same as videos directory')
        
        if path_to_predictions and os.path.isdir(path_to_predictions):
            self.path_to_predictions = os.path.abspath(path_to_predictions)
        else:
            self.path_to_predictions = self.path
            print('path to predictions is invalid, choosing same as videos directory')
            
    def load_frames(self):
        for video in self.videos:
            print(video)
            video_name = video.split('.')[0]
            street = StreetData(os.path.join(self.path_to_frames,video_name))
            street.load_images_from_list()
            if street.images == []:
                street.load_images_from_list()
            if street.images == []:
                print('NO frames found for'+ video)
            else:
                print(len(street.images), ' frames found for '+ video)
#                 street.save(os.path.join(self.path_to_predictions, video_name))

    def load_preds(self):
        streets = {}
        for video in self.videos:
            print(video)
            predictions = []
            video_name = video.split('.')[0]
            street = StreetData(os.path.join(self.path_to_frames, video_name), os.path.join(self.path_to_frames, video_name+'/'+video_name+'.csv'), os.path.join(self.path_to_predictions, video_name+'/'+video_name+'.json'))
            print(street.images_list, street.images_json)
            street.load_data_from_json()
#             images.append(street.images)
#             predictions += street.predictions
            streets[video_name] = {'images': street.images, 'predictions': street.predictions}
        return streets

In [ ]:
ubon = City('/media/ryzen/DATA/ubon/videos', '/media/ryzen/DATA/ubon/frames', '/media/ryzen/DATA/ubon/predictions_v2')

In [ ]:
p = ubon.load_preds()

In [ ]:
import json
json.dump(p, open("/media/ryzen/DATA/ubon/ubon_hotline.json", "w"))

In [ ]:
for j in p:
    for i in p[j]['images']:
        print(i)

In [ ]:
plastic = 0
pile = 0
trashbin = 0
mask = 0
for j in p:
    for i in j:
        if i['Plastic']:
            plastic += i['Plastic']
        if i['Pile']:
            pile += i['Pile']
        if i['Trash bin']:
            trashbin += i['Trash bin']
        if i['Face mask']:
            mask += i['Face mask']

print('Plastic: ', plastic, 'Pile: ', pile, 'Trash bin: ', trashbin, 'Face mask: ', mask)

In [ ]:
for j in p:
    print(len(j))

In [ ]:
m = None
import folium
from folium.plugins import HeatMap, FastMarkerCluster
from folium.features import ColorLine

# get color and opacity

def get_color(val):

    valitudes = {
        0: '#008800',
        1: '#5d8800',
        5: '#7d8800',
        10: '#887600',
        20: '#885800',
        30: '#883f00',
        50: '#883900',
        70: '#882700',
        90: '#881000',
        100: '#880000',
    }
    first = -1
    for valitude in valitudes.keys():
        if first < val <= valitude:
            return valitudes[valitude]
        first = valitude
    return '#FF0000'

icon_circle = folium.plugins.BeautifyIcon(
    icon_shape='circle-dot', 
    border_color='green', 
    border_width=10,
)

# icon_bin=folium.Icon(color="blue",icon="trash", prefix='fa')

# icon_pile=folium.Icon(color="red",icon="dumpster-fire", prefix='fa')

m = folium.Map(tiles="OpenStreetMap", prefer_canvas=True, max_zoom = 25)

images = folium.FeatureGroup(name='Images',overlay=True)
hotline = folium.FeatureGroup(name='Hot line',overlay=True)
heatmap = folium.FeatureGroup(name='Heat map',overlay=True)
piles = folium.FeatureGroup(name='Piles',overlay=True)
trashbins = folium.FeatureGroup(name='Trash bins',overlay=True)

images.add_to(m)
hotline.add_to(m)
heatmap.add_to(m)
piles.add_to(m)
trashbins.add_to(m)

# folium.LayerControl().add_to(m) 

images_points = []

heat_data = []
max_count = 0
for points in p:
    print(len(points))
    for k in range(len(points)):
#         images_points.append([float(points[k]['latitude']), float(points[k]['longitude'])])
#         if points[k]['Trash bin']:
#             folium.Marker([float(points[k]['latitude']), float(points[k]['longitude'])], icon=folium.Icon(color="blue")).add_to(trashbins)
#         if points[k]['Pile']:
#             folium.Marker([float(points[k]['latitude']), float(points[k]['longitude'])], icon=folium.Icon(color="red")).add_to(piles)
#         folium.CircleMarker([float(points[k]['latitude']), float(points[k]['longitude'])], radius=.1, popup="<i>"+points[k]['name']+"</i>", tooltip=points[k]['name'], icon=icon_circle).add_to(images)
        if k < len(points)-1:
            p1_count = points[k]['Plastic']+points[k]['Face mask']+points[k]['Pile']*100
            p2_count = points[k+1]['Plastic']+points[k+1]['Face mask']+points[k+1]['Pile']*100
            avg_count = (p1_count+p2_count)//2
            if max_count < avg_count:
                max_count = avg_count
            color = get_color((p1_count+p2_count)//2)
            heat_data.append([points[k]['latitude'], points[k]['longitude'], avg_count])
#             print(points[k]['latitude'], points[k]['longitude'], points[k+1]['latitude'], points[k+1]['longitude'])
            ColorLine(
                [
                    [points[k]['latitude'], points[k]['longitude']],
                    [points[k+1]['latitude'], points[k+1]['longitude']],
                ],
                colors=[1],
                colormap=[color, color],
                weight=3,
            ).add_to(m)

HeatMap(heat_data, radius = 10, min_opacity = 0.1, max_val = max_count, gradient={.0: 'green', .1: 'yellow', .5: 'red', .9: 'blue'}).add_to(heatmap)
#     break
# FastMarkerCluster(data=images_points, icon_create_function=None).add_to(images)
m.fit_bounds(m.get_bounds())

In [ ]:
m

In [ ]:
m

In [ ]:
m.save('/media/ryzen/DATA/ubon_fastimages.html')

In [ ]:
m

In [ ]:
import os, csv
from exif import Image

def convert2decimal(coords, ref):
        decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
        if ref == "S" or ref == "W":
            decimal_degrees = -decimal_degrees
        return decimal_degrees


for video in os.listdir('/media/ryzen/DATA/ubon/frames'):
    if not video in ['GH010028', 'GH010029', 'GH010030', 'GH010031', 'GH010032', 'GH010033', 'GH010034', 'GH010035', 'GH010036', 'GH010037', 'GH010038', 'GH010039', 'GH010041', 'GH010042', 'GH010043', 'GH010044', 'GH010045', 'GH010047', 'GH010048', 'GH010049', 'GH010051', 'GH010052', 'GH010053', 'GH010054']:
        print(video)
        data = [['Name','Latitude','Longitude']]
        images = os.listdir('/media/ryzen/DATA/ubon/frames/'+video)
        images.sort(key=lambda f: int("".join(filter(str.isdigit, f))))
        for image in images:
            if image.endswith('.jpg'):
                img_exif = Image(open('/media/ryzen/DATA/ubon/frames/'+video+'/'+image, 'rb'))
                if img_exif.has_exif:
                    if 'gps_longitude' in img_exif.list_all() and 'gps_latitude' in img_exif.list_all():
                        row = []
                        row.append(image)
                        row.append(round(convert2decimal(img_exif.gps_latitude, img_exif.gps_latitude_ref), 7))
                        row.append(round(convert2decimal(img_exif.gps_longitude, img_exif.gps_longitude_ref), 7))
                        data.append(row)
    
        with open('/media/ryzen/DATA/ubon/frames/'+video+'/'+video+'.csv', 'w') as csvf:
            writer = csv.writer(csvf)
            for row in data:
                writer.writerow(row)


In [ ]:
import os, csv, json

for video in os.listdir('/media/ryzen/DATA/ubon/frames'):
    if video == 'GH010036-002':
        continue
    print(video)
    data = {}
    data['images'] = []
    data['predictions'] = []
    if not os.path.isfile('/media/ryzen/DATA/ubon/frames/'+video+'/'+video+'.csv'):
        print('no file found')
        continue
    with open('/media/ryzen/DATA/ubon/frames/'+video+'/'+video+'.csv', 'r') as csvf:
        reader = csv.DictReader(csvf)
        for row in reader:
            image = {'name': row['Name'], 'latitude': float(row['Latitude']), 'longitude': float(row['Longitude']), 'Plastic': 0, 'Pile': 0, 'Trash bin': 0, 'Face mask': 0}
            data['images'].append(image)
    with open('/media/ryzen/DATA/ubon/predictions_v2/'+video+'/'+video+'.csv', 'r') as csvp:
        reader = csv.reader(csvp)
        for row in reader:
            if row:
#                 print(row)
                name = row[0]
                cat = row[1]
                x1 = row[2]
                y1 = row[3]
                x2 = row[4]
                y2 = row[5]
                score = row[6]
                image = next((im for im in data['images'] if im['name'] == name), False)
                if not image:
                    print(name)
                    break
                image[cat] += 1
                data['predictions'].append({'image': name, 'class': cat, 'box': [x1, y1, x2, y2], 'score': score})
    with open('/media/ryzen/DATA/ubon/predictions_v2/'+video+'/'+video+'.json', 'w') as jsf:
        json.dump(data, jsf)

In [ ]:
for video in os.listdir('/media/ryzen/DATA/ubon/predictions_v2'):
    with open('/media/ryzen/DATA/ubon/predictions_v2/'+video+'/'+video+'.csv', 'r') as csvf:
        reader = csv.DictReader(csvf)
        a = []
        for row in reader:
            a.append(row)
        if a == []:
            print(video)

In [ ]:
if not '':
    print(1)